# 🧠 Qwen3-1.7B — Chat with Real-Time Thinking

Run all cells to launch a web UI. A **public link** will appear at the bottom — click it to start chatting!

> **Requirements**: Google Colab with **T4 GPU** runtime (free tier).

In [6]:
# Cell 1 — Install Dependencies
!pip install -q transformers accelerate gradio torch bitsandbytes

In [7]:
# Cell 2 — Load Model & Tokenizer
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer, BitsAndBytesConfig

# Clear any leftover GPU memory from previous runs
gc.collect()
torch.cuda.empty_cache()

MODEL_ID = "Qwen/Qwen3-1.7B"

# 4-bit quantization to save GPU memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)

print(f"⏳ Loading tokenizer for {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print(f"⏳ Loading model {MODEL_ID} with 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)

print(f"✅ Model loaded! GPU memory used: {torch.cuda.memory_allocated()/1024**3:.1f} GB")

⏳ Loading tokenizer for Qwen/Qwen3-1.7B...
⏳ Loading model Qwen/Qwen3-1.7B with 4-bit quantization...


Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


✅ Model loaded! GPU memory used: 2.6 GB


In [8]:
# Cell 3 — Chat Logic with Thinking Parser
import threading
import re

def respond(message, history, enable_thinking=True):
    """
    Generator that streams the model's response.
    Parses <think>...</think> tags to separate thinking from the answer.
    Yields (thinking_text, answer_text) tuples as tokens arrive.
    """
    # Build conversation messages
    messages = []
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        if bot_msg:
            messages.append({"role": "assistant", "content": bot_msg})
    messages.append({"role": "user", "content": message})

    # Apply chat template with thinking enabled
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=enable_thinking,
    )
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Set up streamer
    streamer = TextIteratorStreamer(
        tokenizer, skip_prompt=True, skip_special_tokens=True
    )

    # Generation config
    gen_kwargs = dict(
        **inputs,
        max_new_tokens=8192,
        streamer=streamer,
        temperature=0.6,
        top_p=0.95,
        top_k=20,
        do_sample=True,
    )

    # Start generation in a separate thread
    thread = threading.Thread(target=model.generate, kwargs=gen_kwargs)
    thread.start()

    # Stream tokens and parse thinking vs answer
    full_response = ""
    thinking_text = ""
    answer_text = ""
    in_thinking = False
    thinking_done = False

    for new_token in streamer:
        full_response += new_token
        
        # If thinking is disabled, treat everything as answer
        if not enable_thinking:
             answer_text = full_response
             yield "", answer_text
             continue

        if not thinking_done:
            # Check if we've entered the thinking block
            if "<think>" in full_response and not in_thinking:
                in_thinking = True

            # Check if thinking block has ended
            if "</think>" in full_response and in_thinking:
                thinking_done = True
                in_thinking = False
                # Extract thinking content
                think_match = re.search(r"<think>(.*?)</think>", full_response, re.DOTALL)
                if think_match:
                    thinking_text = think_match.group(1).strip()
                # Extract answer (everything after </think>)
                answer_text = full_response.split("</think>")[-1].strip()
                yield thinking_text, answer_text
                continue

            if in_thinking:
                # Extract partial thinking (remove the <think> tag)
                partial = full_response.split("<think>")[-1].strip()
                thinking_text = partial
                yield thinking_text, ""
            else:
                # Before <think> tag appears, show as answer
                answer_text = full_response.strip()
                yield "", answer_text
        else:
            # After thinking is done, everything is the answer
            answer_text = full_response.split("</think>")[-1].strip()
            yield thinking_text, answer_text

    thread.join()

print("✅ Chat logic ready.")

✅ Chat logic ready.


In [9]:
# Cell 4 — Gradio Web UI
import gradio as gr

CUSTOM_CSS = """
/* ── Global ── */
.gradio-container {
    max-width: 900px !important;
    margin: 0 auto !important;
    font-family: 'Inter', 'Segoe UI', sans-serif !important;
}

/* ── Title ── */
.title-text {
    text-align: center;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-size: 2rem;
    font-weight: 800;
    margin-bottom: 0;
}

/* ── Thinking Panel ── */
.thinking-box {
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%) !important;
    border: 1px solid #334155 !important;
    border-radius: 12px !important;
    padding: 16px !important;
    font-size: 0.9rem !important;
    line-height: 1.6 !important;
    color: #94a3b8 !important;
    max-height: 350px !important;
    overflow-y: auto !important;
}

/* ── Answer Panel ── */
.answer-box {
    background: linear-gradient(135deg, #0f172a 0%, #1e293b 100%) !important;
    border: 1px solid #475569 !important;
    border-radius: 12px !important;
    padding: 16px !important;
    font-size: 1rem !important;
    line-height: 1.7 !important;
    color: #e2e8f0 !important;
    min-height: 80px !important;
}

/* ── Chat History ── */
.chat-history {
    border: 1px solid #334155 !important;
    border-radius: 12px !important;
    max-height: 400px !important;
    overflow-y: auto !important;
}

/* ── Input Box ── */
.input-box textarea {
    border-radius: 12px !important;
    border: 1px solid #475569 !important;
    font-size: 1rem !important;
}

/* ── Buttons ── */
.send-btn {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    border: none !important;
    border-radius: 12px !important;
    color: white !important;
    font-weight: 600 !important;
    min-height: 45px !important;
}
.send-btn:hover {
    opacity: 0.9 !important;
    transform: translateY(-1px);
}
.clear-btn {
    border-radius: 12px !important;
    min-height: 45px !important;
}
"""

def user_submit(message, history):
    """Add user message to history and clear input."""
    history = history + [[message, None]]
    return "", history

def bot_respond(history, enable_thinking):
    """
    Generator: streams thinking + answer and updates Gradio components live.
    """
    if not history:
        yield history, "", ""
        return

    user_message = history[-1][0]
    # Build clean history (exclude latest)
    clean_history = [(h[0], h[1] or "") for h in history[:-1]]

    thinking = ""
    answer = ""

    for thinking_chunk, answer_chunk in respond(user_message, clean_history, enable_thinking):
        thinking = thinking_chunk
        answer = answer_chunk

        # Update bot message in history with the answer
        history[-1][1] = answer if answer else "⏳ Thinking..."

        # Format thinking display
        thinking_display = ""
        if thinking:
            thinking_display = f"💭 **Model's Reasoning:**\n\n{thinking}"

        yield history, thinking_display, answer

    # Final update
    if not answer:
        history[-1][1] = thinking if thinking else "(No response generated)"
    else:
        history[-1][1] = answer

    thinking_display = ""
    if thinking:
        thinking_display = f"💭 **Model's Reasoning:**\n\n{thinking}"

    yield history, thinking_display, answer


with gr.Blocks(
    theme=gr.themes.Soft(primary_hue="purple", neutral_hue="slate"),
    css=CUSTOM_CSS,
    title="Qwen3 Chat",
) as demo:

    gr.HTML("<h1 class='title-text'>🧠 Qwen3-1.7B Chat</h1>")
    gr.HTML("<p style='text-align:center;color:#94a3b8;margin-top:-8px;'>Chat with real-time thinking · Powered by Qwen3</p>")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                label="💬 Conversation",
                height=420,
                elem_classes=["chat-history"],
                show_copy_button=True,
            )
            with gr.Row():
                msg_input = gr.Textbox(
                    placeholder="Type your message here... (Enter to send)",
                    show_label=False,
                    scale=5,
                    elem_classes=["input-box"],
                    lines=1,
                )
                send_btn = gr.Button("Send ▶", scale=1, elem_classes=["send-btn"])
                clear_btn = gr.Button("🗑️ Clear", scale=1, elem_classes=["clear-btn"])
                enable_thinking_chk = gr.Checkbox(label="Thinking", value=True, interactive=True, scale=1)
            

        with gr.Column(scale=2):
            with gr.Accordion("🔍 Thinking Process", open=True):
                thinking_display = gr.Markdown(
                    value="*Thinking will appear here in real-time as the model reasons...*",
                    elem_classes=["thinking-box"],
                )
            with gr.Accordion("📝 Final Answer", open=True):
                answer_display = gr.Markdown(
                    value="*The model's answer will appear here...*",
                    elem_classes=["answer-box"],
                )

    # ── Event Handlers ──
    # Submit on Enter
    msg_input.submit(
        fn=user_submit,
        inputs=[msg_input, chatbot],
        outputs=[msg_input, chatbot],
    ).then(
        fn=bot_respond,
        inputs=[chatbot, enable_thinking_chk],
        outputs=[chatbot, thinking_display, answer_display],
    )

    # Submit on button click
    send_btn.click(
        fn=user_submit,
        inputs=[msg_input, chatbot],
        outputs=[msg_input, chatbot],
    ).then(
        fn=bot_respond,
        inputs=[chatbot, enable_thinking_chk],
        outputs=[chatbot, thinking_display, answer_display],
    )

    # Clear button
    clear_btn.click(
        fn=lambda: ([], "*Thinking will appear here...*", "*Answer will appear here...*"),
        outputs=[chatbot, thinking_display, answer_display],
    )

print("🚀 Launching Gradio UI...")
demo.launch(share=True, debug=False)

/tmp/ipython-input-1928717394.py:129: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-1928717394.py:129: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-1928717394.py:140: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-1928717394.py:140: DeprecationWarning: The 'show_copy_button' parameter will be removed in Gradio 6.0. You will need to use 'buttons=["copy"]' instead.
  chatbot = gr.Chatbot(
/tmp/ipython-inp

🚀 Launching Gradio UI...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cbc2c1dac4eef83ac4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---
## ⛔ Stop the Chatbot
Run the cell below to **stop** the chatbot server and free GPU memory.

In [10]:
# Cell 5 — Stop the chatbot server
demo.close()
print("🛑 Chatbot server stopped.")
print("💡 To restart, just re-run Cell 4 above.")

Closing server running on port: 7860
🛑 Chatbot server stopped.
💡 To restart, just re-run Cell 4 above.
